In [1]:
from __future__ import print_function
from collections import deque
from tensorflow.python.framework import ops
from rl_reinforce import REINFORCEothello
from othello_net import *
from othello_rules import *
from feature_extractor import *
import os, random, sys, gym
import tensorflow as tf
import numpy as np
from datetime import datetime
from example_states import *
from training_utils import *
np.set_printoptions(precision=2)
ops.reset_default_graph()

In [10]:
def sample_action(prediction, player):
    prediction = np.transpose(prediction[0])
    prediction = np.transpose(prediction[1])
    legal_moves = find_legal_moves(board, player)
    prediciton = prediction / np.sum(prediction)
    cleaned_predictions = zero_illegal_moves(prediction, legal_moves)
    if player == graph_1 and debug:
        print("cleaned predictions")
        print(cleaned_predictions)
    p = cleaned_predictions.flatten()
    p = p / np.sum(p)
    sample_index = np.flatnonzero( np.random.multinomial(1,p,1) )[0]
    sampled_move = moves[sample_index]
    if prev_move == sampled_move:
        print("action probabilities")
        print(p)
        print(prediction)
        print(cleaned_predictions)
    return sampled_move

moves = ['0'] * 64
for i in range(8):
    for j in range(8):
        moves[i*8 + j] = str((i+1) * 10 + (j+1))

ops.reset_default_graph()

graph_1, img_data_1, train_step_1, optimizer_1, ground_truths_1, loss_1, pred_up_1, keep_prob_1, learn_rate_1, score_out_1 = create_othello_net()
sess_1 = tf.Session(graph=graph_1)
saver_1 = tf.train.Saver()
init_op_1 = tf.initialize_all_variables()
sess_1.run(init_op_1)
model_1 = "models/rl-p-a/a_player_0.ckpt"
if os.path.isfile(model_1):
    saver_1.restore(sess_1, model_1)

graph_2, img_data_2, train_step_2, optimizer_2, ground_truths_2, loss_2, pred_up_2, keep_prob_2, learn_rate_2, score_out_2 = create_othello_net()
sess_2 = tf.Session(graph=graph_2)
saver_2 = tf.train.Saver()
init_op_2 = tf.initialize_all_variables()
sess_2.run(init_op_2)
model_2 = 'models/rl-p-b/b_player_0.ckpt'
if os.path.isfile(model_2):
    saver_2.restore(sess_2, model_2)

tf.reset_default_graph()

graph3, img_data3, train_step3, optimizer3, ground_truths3, loss3, pred_up3, keep_prob3, learn_rate3, score_out3 = create_value_net()
saver3 = tf.train.Saver()
sess3 = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init_op3 = tf.initialize_all_variables()
sess3.run(init_op3)
current_model3 = "models/sl-v/players_g.ckpt.b"
if os.path.isfile(current_model3):
    saver3.restore(sess3, current_model3)
    
        
batches = 1

debug = False
mini = True
use_value_network = True
random_player = False

N = 1 if debug else 100
print("starting self play")


for batch in range(batches):
    graph_1 = -1
    graph_2 = 1
    graph_1_wins = 0
    graph_2_wins = 0
    for n in range(N):
        if n%2 is 0:
            print(n)
        board = initialize_game()
        player = -1
        prev_move = '00'
        while True:
            legal_moves = find_legal_moves(board, player)
            if len(legal_moves) == 0:
                winner = get_winner(board, 1, -1)
                if winner is graph_1:
                    print("graph_1 wins")
                    graph_1_wins += 1
                if winner is graph_2:
                    print("graph_2 wins")
                    graph_2_wins += 1
                break
            features = board_to_input(board, player)
            if player is graph_1:
                prediction = sess_1.run(pred_up_1, feed_dict={img_data_1:[features], keep_prob_1:1.0})
                values = sess3.run(score_out3, feed_dict={img_data3:[features], keep_prob3:1.0})
                values = values[0][0][0]
                soft_values = list_softmax(values)
                if debug:
                    print("board is")
                    print(board)
                    print("soft values is ")
                    print(soft_values)
                    print("moves are")
                    print(legal_moves)
                best_max_value = 0
                best_max_move = random.choice(legal_moves)
                # Try one "period" of minimax
                for max_move in legal_moves:
                    max_board = make_move(board, max_move, player, debug=True)

                    if mini:
                        min_legal_moves = find_legal_moves(max_board, player * (-1))
                        try:
                            best_min_move = random.choice(min_legal_moves)
                        except:
                            print("error:" + str(min_legal_moves))
                        best_min_value = 1
                        for min_move in min_legal_moves:
                            min_board = make_move(max_board, min_move, player * (-1), debug=True)
                            min_features = board_to_input(min_board, player)
                            values = sess3.run(score_out3, feed_dict={img_data3:[min_features], keep_prob3:1.0})
                            values = values[0][0][0]
                            soft_values = list_softmax(values)
                            if player * (-1) is -1:
                                if soft_values[0] < best_min_value:
                                    best_min_value = soft_values[0]
                                    best_min_move = min_move
                            else:
                                if soft_values[1] < best_min_value:
                                    best_min_value = soft_values[1]
                                    best_min_move = min_move
                        if best_max_value < best_min_value:
                            best_max_value = best_min_value
                            best_max_move = max_move
                    else:
                        max_features = board_to_input(max_board, player)
                        values = sess3.run(score_out3, feed_dict={img_data3:[max_features], keep_prob3:1.0})
                        values = values[0][0][0]
                        soft_values = list_softmax(values)
                        if debug:
                            print("move " + str(max_move) + " would have values " + str(soft_values))
                        if soft_values[1] > best_max_value:
                            best_max_value = soft_values[1]
                            best_max_move = max_move
                if use_value_network:
                    sampled_move = best_max_move
                elif random_player:
                    sampled_move = random.choice(leg)
                else:
                    sampled_move = sample_action(prediction, player)
            else:
                prediction = sess_2.run(pred_up_2, feed_dict={img_data_2:[features], keep_prob_2:1.0})
                sampled_move = sample_action(prediction, player)
            
            #print("best move for board is " + str(best_move) + " while sampled move is " + str(sampled_move))
            if prev_move == sampled_move:
                print("illegal action sampled:")
                print(sampled_move)
                print("in state")
                print(board)
                break
                
            prev_move = sampled_move
            board = make_move(board, sampled_move, player, debug=True)

            if player is 1:
                player = -1
            else:
                player = 1
            legal_moves = find_legal_moves(board, player)
            if len(legal_moves) == 0:
                if player is 1:
                    player = -1
                else:
                    player = 1

        graph_1 = graph_1 * (-1)
        graph_2 = graph_2 * (-1)
        if n+1 is N:
            print('%s  %s wr: %.2f, %s wr: %.2f' % (datetime.now().strftime("%d. %H:%M:%S"), model_1[14:], graph_1_wins/float(N), model_2[14:], graph_2_wins/float(N)))
            graph_1_wins = 0
            graph_2_wins = 0


starting self play
0
error:[]
error:[]
error:[]
error:[]
graph_1 wins
error:[]
graph_1 wins
2
error:[]
error:[]
graph_1 wins
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
graph_1 wins
4
error:[]
error:[]
graph_1 wins
error:[]
error:[]
error:[]
error:[]
error:[]
graph_1 wins
6
error:[]
error:[]
error:[]
error:[]
graph_1 wins
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
graph_1 wins
8
graph_1 wins
error:[]
error:[]
error:[]
graph_1 wins
10
error:[]
error:[]
graph_1 wins
error:[]
error:[]
error:[]
error:[]
error:[]
graph_1 wins
12
error:[]
error:[]
error:[]
graph_2 wins
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
graph_1 wins
14
error:[]
error:[]
graph_1 wins
error:[]
error:[]
error:[]
graph_1 wins
16
error:[]
error:[]
graph_1 wins
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]
error:[]


In [ ]:
# MINIMAX: 15. 17:44:39  a_player_0.ckpt wr: 0.93, b_player_0.ckpt wr: 0.05

In [3]:
def create_value_net():
    with tf.variable_scope("value_network"):
        img_data = tf.placeholder(tf.float32, shape=[None, 8, 8, 28], name="img_data")
        keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        #convolutional layers 
        i = 28; o = 32;
        conv1_weights = conv_weights(i, o)
        conv1_biases = conv_biases(o)
        conv1_out = conv_nonparams2(img_data, conv1_weights, conv1_biases, keep_prob)
        #pool1 = max_pool_2x2(conv1_out)
        i = o; o = 32;
        conv2_weights = conv_weights(i, o)
        conv2_biases = conv_biases(o)
        conv2_out = conv_nonparams2(conv1_out, conv2_weights, conv2_biases, keep_prob)
        #pool2 = max_pool_2x2(conv2_out)
        i = o; o = 32;
        conv3_weights = conv_weights2(i, o)
        conv3_biases = conv_biases(o)
        conv3_out = conv_nonparams2(conv2_out, conv3_weights, conv3_biases, keep_prob)
        
        i = o; o = 32;
        conv4_weights = conv_weights2(i, o)
        conv4_biases = conv_biases(o)
        conv4_out = conv_nonparams2(conv3_out, conv4_weights, conv4_biases, keep_prob)
        

        
        k = 1; i = o; o = 2;
        s_weights = weight_variable([k, k, i, o])
        s_biases = bias_variable([o])
        s_out = conv_nonparams(conv4_out, s_weights, s_biases, keep_prob)
        
        k = 8; i = o; o = 2;
        score_weights = weight_variable([k, k, i, o])
        score_biases = bias_variable([o])
        score_out = conv_nonparams3(s_out, score_weights, score_biases, keep_prob)
        score_out_flat = tf.reshape(score_out, [-1, o])
        #final layer
        #Don't actually use this layer here
        predictions = softmax(score_out, o)

        #training block:
        label = tf.placeholder(tf.float32, shape=[None, 2], name="ground_truths")
        learn_rate = tf.placeholder(tf.float32, name="eta")
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(score_out_flat, label))
        policy_network_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        reg_loss = tf.reduce_sum([tf.reduce_sum(tf.square(x)) for x in policy_network_variables])
        optimizer = tf.train.AdamOptimizer(learn_rate)
        train_step = optimizer.minimize(loss)
        sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        return sess.graph, img_data, train_step, optimizer, label, loss, predictions, keep_prob, learn_rate, score_out
    
def list_softmax(x):
    """Compute softmax values for each sets of scores in x."""
    sf = np.exp(x)
    sf = sf/np.sum(sf, axis=0)
    return sf

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def conv_nonparams2(tensor_input, conv_weights, conv_biases, keep_prob):
    conv_in = conv(tensor_input, conv_weights)
    conv_relu = lrelu(conv_in + conv_biases)
    conv_drop = tf.nn.dropout(conv_relu, keep_prob)
    return conv_drop

def conv_weights2(i, o):
    k = 2
    shape = [k, k, i, o]
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def conv_nonparams3(tensor_input, conv_weights, conv_biases, keep_prob):
    conv_in = conv(tensor_input, conv_weights, "VALID")
    conv_relu = lrelu(conv_in + conv_biases)
    conv_drop = tf.nn.dropout(conv_relu, keep_prob)
    return conv_drop